## Finilizing Data Generation for 0D Reactor

In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
WORKSPACE_PATH  = os.environ['WORKSPACE_PATH']
plt.style.use(WORKSPACE_PATH+'/ROMNet/romnet/extra/postprocessing/presentation.mplstyle')

from scipy.integrate import solve_ivp

pd.options.mode.chained_assignment = None  # default='warn'

from PCAfold import PCA as PCAA

3.8.12 (default, Aug 31 2021, 04:09:21) 
[Clang 12.0.5 (clang-1205.0.22.9)]


In [2]:
### Input Data

OutputDir  = WORKSPACE_PATH+'/ROMNet/Data/0DReact_Isobaric_100Cases_POD/'
# TestDir    = WORKSPACE_PATH+'/ROMNet/Data/0DReact_Isobaric_100Cases_Test/'
try:
    os.makedirs(OutputDir)
except OSError as e:
    pass
# try:
#     os.makedirs(TestDir)
# except OSError as e:
#     pass

NVarsRed           = 7

tStratch           = 1.
SOLVER             = 'BDF'

ValidPerc          = 25.

FixedMinVal        = 1.e-14

POD_NAME           = '1'
iPC                = 0

if (POD_NAME == 'All'):
    NPODsFinal = 128
    PODType    = 'All'
else:
    NPODsFinal = 64
    PODType    = 'OneByOne'
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType))
except:
    pass

In [3]:
FileName     = OutputDir + '/Orig/train/ext/SimIdxs.csv'
Data         = pd.read_csv(FileName, header=0)
SimIdxs      = Data.to_numpy(int)


FileName     = OutputDir + '/Orig/train/ext/ICs.csv'
Data         = pd.read_csv(FileName)
ICVecs       = Data.to_numpy()
NICs         = len(ICVecs)

In [4]:
if (POD_NAME == 'All'):
    
    Cols   = []
    NewFlg = True
    for iC in range(NICs):

        Nts                  = SimIdxs[iC,1]-SimIdxs[iC,0]
        NtsNow               = Nts



        FileName             = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
        Data                 = pd.read_csv(FileName, header=0)
        DataTemp             = Data[['PC_'+str(iPC+1) for iPC in range(NVarsRed)]]


        for iPC in range(NVarsRed):
            if (np.abs( (DataTemp['PC_'+str(iPC+1)][0] - DataTemp['PC_'+str(iPC+1)][len(Data)-1])/DataTemp['PC_'+str(iPC+1)][0] ) > 1.e-3): 
                if (NewFlg):
                    DataInput        = DataTemp
                    NewFlg           = False
                else:
                    DataInput        = pd.concat([DataInput, DataTemp['PC_'+str(iPC+1)]], axis=1)
                #Cols.append(str(iC+1))
                Cols = Cols+[str(iC+1)+'_PC_'+str(iPC+1)]

    tVec              = Data['t']
    
    yMat              = DataInput.to_numpy()
    
else:
    
    Cols   = []
    NewFlg = True
    for iC in range(NICs):

        Nts                  = SimIdxs[iC,1]-SimIdxs[iC,0]
        NtsNow               = Nts



        FileName             = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
        Data                 = pd.read_csv(FileName, header=0)
        DataTemp             = Data['PC_'+str(iPC+1)]

        if (np.abs( (DataTemp[0] - DataTemp[len(Data)-1])/DataTemp[0] ) > 1.e-3):
            DataICTemp  = Data[['PC_'+str(iPC+1) for iPC in range(NVarsRed)]].iloc[0]
            
            if (NewFlg):
                DataInput        = DataTemp
                DataIC           = DataICTemp
                NewFlg           = False
            else:
                DataInput        = pd.concat([DataInput, DataTemp], axis=1)
                DataIC           = pd.concat([DataIC, DataICTemp], axis=1)
            Cols.append(str(iC+1))

    tVec              = Data['t']
    DataInput.columns = Cols

    yMat              = DataInput.to_numpy()
    
    DataIC            = DataIC.T.reset_index(drop=True, inplace=False)

In [5]:
ICs = DataIC.to_numpy()

In [6]:
DataIC

,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7
0,0.622647,-0.016205,-0.082736,-0.010019,0.042125,-0.100087,0.135381
1,0.676314,-0.241883,-0.113379,0.086361,0.080833,-0.077690,0.071159
2,0.640750,-0.142490,-0.109011,0.039637,0.072141,-0.105261,0.120543
3,0.707118,-0.032441,-0.023253,0.025810,-0.011558,0.021196,-0.011832
4,0.682497,-0.182818,-0.089878,0.068388,0.056532,-0.053519,0.052183
...,...,...,...,...,...,...,...
495,0.753656,0.103028,0.055409,-0.003815,-0.089961,0.124667,-0.112590
496,0.632699,-0.046922,-0.084804,0.004084,0.045470,-0.092960,0.121781
497,0.652726,0.056227,-0.036701,-0.023996,-0.003435,-0.037049,0.072317
498,0.753561,0.070478,0.044992,0.007277,-0.078889,0.116264,-0.107992


In [7]:
# NPODs      = 128
# ScalingVec = ['none', 'auto', 'std', 'pareto', 'vast', 'range', '0to1']
# #'none', '', 'auto', 'std', 'pareto', 'vast', 'range', '0to1', '-1to1', 'level', 'max', 'poisson', 'vast_2', 'vast_3', 'vast_4'

# plt.figure(figsize=(8,8))
# for j, Scaling in enumerate(ScalingVec):

#     pca        = PCAA(yMat, scaling=Scaling, n_components=int(NPODs))
#     C          = pca.X_center
#     D          = pca.X_scale
#     A          = pca.A[:,0:NPODs].T
#     L          = pca.L
#     AT         = A.T

#     plt.plot(L, label=Scaling)
    
# plt.ylim([1.e-14, 1.e0])
# plt.yscale('log')
# plt.legend()

In [8]:
# NPODsVec   = np.arange(30,150,10)
# ScalingVec = ['none', 'auto', 'std', 'pareto', 'vast', 'range', '0to1']
# #'none', '', 'auto', 'std', 'pareto', 'vast', 'range', '0to1', '-1to1', 'level', 'max', 'poisson', 'vast_2', 'vast_3', 'vast_4'

# MaxPerc    = np.zeros((len(NPODsVec), len(ScalingVec)))
# MaxSE      = np.zeros((len(NPODsVec), len(ScalingVec)))
# MeanPerc   = np.zeros((len(NPODsVec), len(ScalingVec)))
# MSE        = np.zeros((len(NPODsVec), len(ScalingVec)))
# for i, NPODs in enumerate(NPODsVec):
#     #print('NPODs = ', NPODs)
    
#     for j, Scaling in enumerate(ScalingVec):

#         pca        = PCAA(yMat, scaling=Scaling, n_components=int(NPODs))
#         C          = pca.X_center
#         D          = pca.X_scale
#         A          = pca.A[:,0:NPODs].T
#         L          = pca.L
#         AT         = A.T

#         yMat_pca   = ((yMat - C)/D).dot(AT)
#         yMat_      = (yMat_pca.dot(A))*D + C
        
#         MaxPerc[i,j]    = np.max(abs((yMat - yMat_)/yMat)*100)
#         MaxSE[i,j]      = np.max((yMat - yMat_)**2)
#         MeanPerc[i,j]   = np.mean(abs((yMat - yMat_)/yMat)*100)
#         MSE[i,j]        = np.mean((yMat - yMat_)**2)

# plt.figure(figsize=(8,8))
# plt.plot(NPODsVec, MaxPerc, label=ScalingVec)
# plt.xlabel('No of PODs')
# plt.ylabel('Max % Error')
# plt.legend()
# plt.yscale('log')

# plt.figure(figsize=(8,8))
# plt.plot(NPODsVec, MaxSE, label=ScalingVec)
# plt.xlabel('No of PODs')
# plt.ylabel('Max Squared Error')
# plt.legend()
# plt.yscale('log')

# plt.figure(figsize=(8,8))
# plt.plot(NPODsVec, MeanPerc, label=ScalingVec)
# plt.xlabel('No of PODs')
# plt.ylabel('Mean % Error')
# plt.legend()
# plt.yscale('log')

# plt.figure(figsize=(8,8))
# plt.plot(NPODsVec, MSE, label=ScalingVec)
# plt.xlabel('No of PODs')
# plt.ylabel('Mean Squared Error')
# plt.legend()
# plt.yscale('log')

In [26]:
pca        = PCAA(yMat, scaling='pareto', n_components=NPODsFinal, nocenter=False)
C          = pca.X_center
D          = pca.X_scale
A          = pca.A[:,0:NPODsFinal].T
L          = pca.L
AT         = A.T
print('[PCA] Shape of A        = ', A.shape)
print('[PCA] ')

[PCA] Shape of A        =  (64, 500)
[PCA] 


In [27]:
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/ROM/')
except:
    pass

FileName    = OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/ROM/A.csv'
np.savetxt(FileName, A, delimiter=',')

FileName    = OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/ROM/C.csv'
np.savetxt(FileName, C, delimiter=',')

FileName    = OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/ROM/D.csv'
np.savetxt(FileName, D, delimiter=',')

In [28]:
#yMat_pca    = pca.transform(yMat, nocenter=False)
yMat_pca   = ((yMat - C)/D).dot(AT)

yMat_      = (yMat_pca.dot(A))*D + C
print('[PCA] Shape of yMat_pca = ', yMat_pca.shape)
print('[PCA]  Max % Error = ', np.max(abs((yMat - yMat_)/yMat)*100))
print('[PCA]  Max      SE = ', np.max((yMat - yMat_)**2))
print('[PCA] Mean % Error = ', np.mean(abs((yMat - yMat_)/yMat)*100))
print('[PCA]          MSE = ', np.mean((yMat - yMat_)**2))

[PCA] Shape of yMat_pca =  (5000, 64)
[PCA]  Max % Error =  71598.65079004323
[PCA]  Max      SE =  0.014988363637289793
[PCA] Mean % Error =  0.2387674305074207
[PCA]          MSE =  2.84442275149419e-06


In [30]:
%matplotlib qt

iVar    = 40

fig = plt.figure()
ax  = fig.add_subplot(projection='3d')
ax.scatter(ICs[:,0], ICs[:,1], A[iVar,:], c='k')
ax.set_xlabel(r'$\eta_{'+str(1)+'}$')
ax.set_ylabel(r'$\eta_{'+str(2)+'}$')
ax.set_zlabel(r'$\Psi_{'+str(iVar+1)+'}$')

Text(0.5, 0, '$\\Psi_{41}$')

In [13]:
A.shape

(64, 500)

In [14]:
plt.figure(figsize=(10,8))
plt.plot(tVec, yMat_pca[:,0], '-k')
plt.xscale('log')
#plt.xlim([1.e-6,1.e-4])

In [15]:
plt.figure(figsize=(10,8))
plt.plot(tVec, yMat[:,0], '-k')
plt.plot(tVec, yMat_[:,0], '-r')
plt.xscale('log')
plt.xlim([1.e-6,1.e-2])

(1e-06, 0.01)

In [16]:
Vars             = ['POD_'+str(iPOD+1) for iPOD in range(NPODsFinal)]
Data             = pd.DataFrame(yMat_pca, columns=Vars)
tVec[tVec == 0.] = FixedMinVal
Data['t']        = tVec
Data['log(t)']   = np.log(tVec)
Data['log10(t)'] = np.log10(tVec)


Data

,POD_1,POD_2,POD_3,POD_4,POD_5,POD_6,POD_7,POD_8,POD_9,POD_10,...,POD_58,POD_59,POD_60,POD_61,POD_62,POD_63,POD_64,t,log(t),log10(t)
0,17.814214,1.898052,-0.622610,0.362329,-0.150731,0.166322,0.090171,-0.079997,0.044110,-0.065767,...,-0.001411,0.000319,0.000695,-0.000065,-0.000800,0.000151,-0.000226,1.000000e-14,-32.236191,-14.000000
1,17.814214,1.898052,-0.622610,0.362329,-0.150731,0.166322,0.090171,-0.079997,0.044110,-0.065767,...,-0.001411,0.000319,0.000695,-0.000065,-0.000800,0.000151,-0.000226,1.000000e-12,-27.631021,-12.000000
2,17.814214,1.898052,-0.622610,0.362329,-0.150731,0.166322,0.090171,-0.079997,0.044110,-0.065767,...,-0.001411,0.000319,0.000695,-0.000065,-0.000800,0.000151,-0.000226,1.097499e-12,-27.537987,-11.959596
3,17.814214,1.898052,-0.622610,0.362329,-0.150731,0.166322,0.090171,-0.079997,0.044110,-0.065767,...,-0.001411,0.000319,0.000695,-0.000065,-0.000800,0.000151,-0.000226,1.204504e-12,-27.444954,-11.919192
4,17.814214,1.898052,-0.622610,0.362329,-0.150731,0.166322,0.090171,-0.079997,0.044110,-0.065767,...,-0.001411,0.000319,0.000695,-0.000065,-0.000800,0.000151,-0.000226,1.321941e-12,-27.351920,-11.878788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,-23.628650,3.623634,1.730064,1.106611,0.614522,0.442135,-0.291566,-0.274149,-0.185658,-0.089890,...,-0.001353,-0.008494,-0.001812,-0.005172,-0.006447,0.005250,0.024701,9.887808e-03,-4.616453,-2.004900
4996,-23.629536,3.624076,1.729606,1.106727,0.614477,0.442061,-0.291685,-0.274057,-0.185571,-0.089888,...,-0.001359,-0.008542,-0.001852,-0.005157,-0.006470,0.005293,0.024809,9.915738e-03,-4.613632,-2.003675
4997,-23.630419,3.624519,1.729149,1.106843,0.614432,0.441988,-0.291805,-0.273965,-0.185483,-0.089885,...,-0.001365,-0.008590,-0.001893,-0.005142,-0.006493,0.005336,0.024915,9.943746e-03,-4.610811,-2.002450
4998,-23.631299,3.624961,1.728693,1.106957,0.614388,0.441915,-0.291925,-0.273873,-0.185393,-0.089883,...,-0.001371,-0.008637,-0.001933,-0.005127,-0.006515,0.005380,0.025021,9.971833e-03,-4.607991,-2.001225


In [17]:
from sklearn.model_selection import train_test_split

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME)
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/valid/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/test/')
except:
    pass

In [18]:
data_id    = 'pts'

DataNoZero           = Data[Data['t'] >= FixedMinVal]
n_points             = len(DataNoZero)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=ValidPerc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)


try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/valid/'+data_id+'/')
except:
    pass

DataInput  = DataNoZero[['t', 'log10(t)', 'log(t)'] + Vars]
DataInput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/valid/'+data_id+'/Input.csv', index=False)

DataOutput = DataNoZero[['t', 'log10(t)', 'log(t)'] + Vars]
DataOutput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/valid/'+data_id+'/Output.csv', index=False)

In [19]:
DataNoZero           = Data
n_points             = len(DataNoZero)

n_test               = n_points


try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/test/ext/')
except:
    pass

DataInput  = DataNoZero[['t', 'log10(t)', 'log(t)'] + Vars]
DataInput.to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/test/ext/Input.csv', index=False)

DataOutput = DataNoZero[['t', 'log10(t)', 'log(t)'] + Vars]
DataOutput.to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Trunk/test/ext/Output.csv', index=False)

In [20]:
from sklearn.model_selection import train_test_split

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME)
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/valid/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/test/')
except:
    pass

In [21]:
data_id    = 'pts'

n_points             = len(DataIC)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=ValidPerc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)


try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/valid/'+data_id+'/')
except:
    pass

DataInput  = DataIC
DataInput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/train/'+data_id+'/Input.csv', index=False)
DataInput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/valid/'+data_id+'/Input.csv', index=False)

DataOutput = pd.DataFrame(A.T, columns=['POD_'+str(i+1) for i in range(A.shape[0])])
DataOutput.iloc[train_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/train/'+data_id+'/Output.csv', index=False)
DataOutput.iloc[valid_idx].to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/valid/'+data_id+'/Output.csv', index=False)

In [22]:
n_points             = len(DataIC)

n_test               = n_points

try:
    os.makedirs(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/test/ext/')
except:
    pass

DataInput  = DataIC
DataInput.to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/test/ext/Input.csv', index=False)

DataOutput = pd.DataFrame(A.T, columns=['POD_'+str(i+1) for i in range(A.shape[0])])
DataOutput.to_csv(OutputDir+'/'+str(NVarsRed)+'PC/'+str(PODType)+'/POD_'+POD_NAME+'/Branch/test/ext/Output.csv', index=False)

In [23]:
NComp = 2

pca        = PCAA(ICs, scaling='none', n_components=NComp)
C          = pca.X_center
D          = pca.X_scale
A          = pca.A[:,0:NComp].T
L          = pca.L
AT         = A.T
print('[PCA] Shape of A        = ', A.shape)
print('[PCA] ')

[PCA] Shape of A        =  (2, 7)
[PCA] 


In [24]:
#yMat_pca    = pca.transform(yMat, nocenter=False)
ICs_pca   = ((ICs - C)/D).dot(AT)

ICs_      = (ICs_pca.dot(A))*D + C
print('[PCA] Shape of yMat_pca = ', ICs_pca.shape)
print('[PCA]  Max % Error = ', np.max(abs((ICs - ICs_)/ICs)*100))
print('[PCA]  Max      SE = ', np.max((ICs - ICs_)**2))
print('[PCA] Mean % Error = ', np.mean(abs((ICs - ICs_)/ICs)*100))
print('[PCA]          MSE = ', np.mean((ICs - ICs_)**2))

[PCA] Shape of yMat_pca =  (500, 2)
[PCA]  Max % Error =  3.985247461657696e-11
[PCA]  Max      SE =  1.9721522630525295e-31
[PCA] Mean % Error =  1.5859021862551634e-13
[PCA]          MSE =  1.1510452812793156e-32


In [25]:
A

array([[-0.09572019,  0.87972368,  0.20629378, -0.33502592, -0.23038227,
         0.08116266,  0.04960883],
       [ 0.36943056, -0.04368819,  0.26882912,  0.14735975, -0.24401961,
         0.53736105, -0.64755381]])